In [0]:
!pip install -q -U keras

from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.pooling import MaxPool2D
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array

from keras.utils import plot_model, np_utils

from keras.datasets import cifar10

import numpy as np

img_rows, img_cols = 32, 32
img_channels = 3
nb_classes = 10

nb_epochs = 100
batch_size = 32

     |████████████████████████████████| 378kB 9.7MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


Using TensorFlow backend.


60000枚の サイズ 32x32 のカラー(RGB: 3チャンネル) の画像データを使用する。
画像は 50000枚の学習データと 10000枚のテストデータに分けられる。

画像は10種類の分類分けがされている。
- 飛行機
- 自動車
- 鳥
- 猫
- 鹿
- 犬
- 蛙
- 馬
- 船
- トラック

各画像がどの分類化を推測するのがゴールである。

In [0]:
(x_train, y_train_nominal),(x_test, y_test_nominal) = cifar10.load_data()
print(x_train.shape, y_train_nominal.shape)
print(x_test.shape, y_test_nominal.shape)

# 左上1マス目の RGBの値
print(x_train[0][0][0])

# floatに型変換
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0


print(y_train_nominal.shape)
print(y_train_nominal[0])

# One-Hot表現に変換
y_train = np_utils.to_categorical(y_train_nominal, nb_classes)
y_test = np_utils.to_categorical(y_test_nominal, nb_classes)

print(y_train.shape)
print(y_train[0])

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)
[59 62 63]
(50000, 1)
[6]
(50000, 10)
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


ネットワーク構築 (VGG16)

- 3x3の畳み込みを何度か繰り返す
- max poolingを1回行い、サイズを半分にする
- pooling後はチャンネル数を倍にする

In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, activation='softmax'))

model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)         

学習

In [0]:
history = model.fit(
    x_train, y_train,
    batch_size=batch_size, epochs=nb_epochs, verbose=1,
    validation_data=(x_test, y_test),
    callbacks=[EarlyStopping(patience=0, verbose=1)]
)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 20s 401us/step - loss: 1.4984 - accuracy: 0.4529 - val_loss: 1.1037 - val_accuracy: 0.6083
Epoch 2/100
50000/50000 [==============================] - 13s 254us/step - loss: 1.0958 - accuracy: 0.6084 - val_loss: 0.9039 - val_accuracy: 0.6819
Epoch 3/100
50000/50000 [==============================] - 12s 249us/step - loss: 0.9304 - accuracy: 0.6733 - val_loss: 0.7934 - val_accuracy: 0.7206
Epoch 4/100
50000/50000 [==============================] - 13s 254us/step - loss: 0.8267 - accuracy: 0.7103 - val_loss: 0.8093 - val_accuracy: 0.7125
Epoch 00004: early stopping


In [0]:
model.save('model.h5', include_optimizer=False)

!ls -l

total 8516
drwx------ 3 root root    4096 Apr  3 11:17 drive
-rw-r--r-- 1 root root 8708296 Apr  3 11:18 model.h5
drwxr-xr-x 1 root root    4096 Mar 18 16:23 sample_data


In [0]:
!ls -l drive/My\ Drive/images/

img = load_img('drive/My Drive/images/0001.png', target_size=(img_rows, img_cols))

x = img_to_array(img).astype('float32')/255.0
print(x.shape)

x = np.expand_dims(x, axis=0)

print(x.shape)
print(x[0])

total 3
-rw------- 1 root root 2151 Feb 17 00:35 0001.png
(32, 32, 3)
(1, 32, 32, 3)
[[[0.60784316 0.6117647  0.58431375]
  [0.654902   0.6901961  0.73333335]
  [0.6901961  0.7019608  0.75686276]
  ...
  [0.7882353  0.76862746 0.81960785]
  [0.7921569  0.7921569  0.83137256]
  [0.7529412  0.7176471  0.67058825]]

 [[0.6        0.60784316 0.6156863 ]
  [0.6392157  0.7019608  0.8       ]
  [0.67058825 0.72156864 0.84313726]
  ...
  [0.7921569  0.8156863  0.87058824]
  [0.79607844 0.84313726 0.8901961 ]
  [0.7411765  0.74509805 0.7176471 ]]

 [[0.60784316 0.6039216  0.6       ]
  [0.627451   0.69803923 0.7882353 ]
  [0.65882355 0.7254902  0.8352941 ]
  ...
  [0.80784315 0.8156863  0.84313726]
  [0.8        0.84313726 0.8784314 ]
  [0.7411765  0.7490196  0.72156864]]

 ...

 [[0.34901962 0.30980393 0.24313726]
  [0.22352941 0.20784314 0.20392157]
  [0.23921569 0.23921569 0.23137255]
  ...
  [0.34901962 0.46666667 0.39607844]
  [0.23137255 0.30980393 0.24313726]
  [0.2627451  0.2901961  0.1

In [0]:
model2 = load_model('model.h5', compile=False)

score = model2.predict(x)

print(score.shape)

print(score)

(1, 10)
[[5.7788086e-01 8.0083735e-02 1.4008910e-02 5.8175595e-03 1.3879163e-02
  6.0552527e-04 2.2595720e-03 3.3509490e-04 2.6851991e-01 3.6609665e-02]]
